In [3]:
%pip install newsapi-python
%pip install --upgrade transformers
%pip install bert-for-sequence-classification
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='cc7b577d5b4b4462b6eef124170903b1')

articles = newsapi.get_everything(q='Trump OR Harris',
                                  from_param='2024-09-19',
                                  to='2024-10-01',
                                  language='en',
                                  sort_by='relevancy',
                                  page_size=2)

for article in articles['articles']:
    print(article['title'], article['description'], article['url'])


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



  Using cached bert_for_sequence_classification-0.1.1-py3-none-any.whl (17 kB)
                                              0.0/199.4 MB ? eta -:--:--
                                             0.4/199.4 MB 11.6 MB/s eta 0:00:18
                                             0.8/199.4 MB 10.2 MB/s eta 0:00:20
                                             1.1/199.4 MB 10.2 MB/s eta 0:00:20
                                             1.7/199.4 MB 10.6 MB/s eta 0:00:19
                                             2.2/199.4 MB 10.0 MB/s eta 0:00:20
                                             2.7/199.4 MB 10.6 MB/s eta 0:00:19
                                             3.1/199.4 MB 10.4 MB/s eta 0:00:19
                                             3.6/199.4 MB 10.5 MB/s eta 0:00:19
                                             3.7/199.4 MB 10.4 MB/s eta 0:00:19
                                              4.3/199.4 MB 9.7 MB/s eta 0:00:21
                                              4.

Trump Says He’ll Prosecute Google If He Retakes Power The former president insists Google is hiding positive news stories about him. https://gizmodo.com/trump-says-hell-prosecute-google-if-he-retakes-power-2000504499
Kamala Harris accepts CNN invitation for TV debate Trump has yet to comment on the proposed rematch, but has said previously he wouldn't debate Harris again. https://www.bbc.com/news/articles/cwyejk91d2qo


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    words = word_tokenize(text.lower())
    words = [WordNetLemmatizer().lemmatize(word) for word in words if word.isalpha()]
    return ' '.join([word for word in words if word not in stop_words])

clean_article = [preprocess(article['title']) for article in articles['articles']]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
for article in clean_article:
    sentiment = sid.polarity_scores(article)
    print(article, sentiment)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...


trump say prosecute google retake power {'neg': 0.351, 'neu': 0.649, 'pos': 0.0, 'compound': -0.4019}
kamala harris accepts cnn invitation tv debate {'neg': 0.0, 'neu': 0.723, 'pos': 0.277, 'compound': 0.3182}
get vp debate bingo card right {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
staged debate test bluetooth earring kamala harris wear {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
kamala harris go offensive border message {'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'compound': -0.4588}
iranian hacker tried give hacked trump campaign email dems {'neg': 0.252, 'neu': 0.748, 'pos': 0.0, 'compound': -0.4019}
lunatic disgrace trump harris describe {'neg': 0.681, 'neu': 0.319, 'pos': 0.0, 'compound': -0.7506}
iranian hacker tried tempt biden campaign stolen trump info {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4939}
watch vp debate cable required {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
trump spread hurricane helene conspiracy hundred remain missin

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

In [13]:
from nltk.corpus import wordnet as wn
import pandas as pd

nrc = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=["word", "emotion", "association"])
nrc = nrc.pivot(index='word', columns='emotion', values='association').reset_index()

def get_emotions(text):
    words = word_tokenize(text.lower())
    emotions = {emotion: 0 for emotion in nrc.columns[1:]}
    for word in words:
        if word in nrc['word'].values:
            word_emotions = nrc[nrc['word'] == word].iloc[0, 1:]
            for emotion in word_emotions.index:
                emotions[emotion] += word_emotions[emotion]
    return emotions

for article in clean_article:
    emotions = get_emotions(article)
    print(article, emotions)


FileNotFoundError: [Errno 2] No such file or directory: 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sentiments = ['positive', 'negative', 'neutral']
counts = [len([s for s in clean_article if sid.polarity_scores(s)['compound'] > 0.5]),
          len([s for s in clean_article if sid.polarity_scores(s)['compound'] < -0.5]),
          len([s for s in clean_article if -0.5 <= sid.polarity_scores(s)['compound'] <= 0.5])]

sns.barplot(x=sentiments, y=counts)
plt.title('Sentiment Distribution for Trump and Harris News Articles')
plt.show()
